<a href="https://colab.research.google.com/github/SP3kz/BERTopic/blob/master/examples/inverse_folding/notebook_multichain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inverse folding with ESM-IF1

The ESM-IF1 inverse folding model is built for predicting protein sequences from their backbone atom coordinates. We provide examples here 1) to sample sequence designs for a given structure and 2) to score sequences for a given structure.

Trained with 12M protein structures predicted by AlphaFold2, the ESM-IF1 model consists of invariant geometric input processing layers followed by a sequence-to-sequence transformer, and achieves 51% native sequence recovery on structurally held-out backbones. The model is also trained with span masking to tolerate missing backbone coordinates and therefore can predict sequences for partially masked structures.

See [GitHub README](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding) for the complete user guide, and see our [bioRxiv pre-print](https://doi.org/10.1101/2022.04.10.487779) for more details.

## Environment setup (colab)
This step might take up to 10 minutes the first time.

If using a local jupyter environment, instead of the following, we recommend configuring a conda environment upon first use in command line:
```
conda create -n inverse python=3.9
conda activate inverse
conda install pytorch cudatoolkit=11.3 -c pytorch
conda install pyg -c pyg -c conda-forge
conda install pip
pip install biotite
pip install git+https://github.com/facebookresearch/esm.git
```

Afterwards, `conda activate inverse` to activate this environment before starting `jupyter notebook`.

Below is the setup for colab notebooks:

We recommend using GPU runtimes on colab (Menu bar -> Runtime -> Change runtime type -> Hardware accelerator -> GPU)

In [ ]:
# Colab environment setup

# Install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 127.2 MB/s eta 0:00:00


### Verify that pytorch-geometric is correctly installed

If the notebook crashes at the import, there is likely an issue with the version of torch_geometric and torch_sparse being incompatible with the torch version.

In [ ]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

## Load model
This steps takes a few minutes for the model to download

In [ ]:
import esm
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
# use eval mode for deterministic output e.g. without random dropout
model = model.eval()

## Load structure from PDB or CIF files

As an example, let's look at Golgi Casein Kinase, the [PDB Molecule of the Month from January 2022](https://pdb101.rcsb.org/motm/265).

Milk is a complex mixture of proteins, fats, and nutrients that provides everything that a growing infant needs. Most of the protein in cow’s milk is casein, whereas human milk has lesser amounts of casein.

The Golgi casein kinase (PDB entry 5YH2) adds phosphates to casein and also to many other types of secreted proteins. It is most active as a complex of two similar types of proteins. Fam20C is the catalytic subunit. It binds to casein and transfers a phosphate from ATP to the protein.

In this example, let's load chains A, B, C, and D, and try to redesign chain A.

You may also upload your own CIF or PDB file and specify the chain id.

In [ ]:
!wget https://files.rcsb.org/download/5YH2.cif -P data/    # save this to the data folder in colab

Load chains A, B, C, D from this CIF file:

In [ ]:
fpath = 'data/5YH2.cif' # .pdb format is also acceptable
chain_ids = ['A', 'B', 'C', 'D']
structure = esm.inverse_folding.util.load_structure(fpath, chain_ids)
coords, native_seqs = esm.inverse_folding.multichain_util.extract_coords_from_complex(structure)

print(f'Loaded chains: {list(coords.keys())}\n')

for chain_id in chain_ids:
    print(f'Chain {chain_id} native sequence:')
    print(native_seqs[chain_id])
    print('\n')

Visualize chain A in this CIF file:

In [ ]:
!pip install -q py3Dmol

In [ ]:
try:
    import py3Dmol

    def view_pdb(fpath, chain_ids, colors=['red','blue','green']):
        with open(fpath) as ifile:
            system = "".join([x for x in ifile])

        view = py3Dmol.view(width=600, height=400)
        view.addModelsAsFrames(system)
        for chain_id, color in zip(chain_ids, colors):
            view.setStyle({'model': -1, 'chain': chain_id}, {"cartoon": {'color': color}})
        view.zoomTo()
        view.show()

except ImportError:
    def view_pdb(fpath, chain_id):
        print("Install py3Dmol to visualize, or use pymol")

In [ ]:
view_pdb(fpath, ['A'])

## Sample sequence and calculate sequence recovery:

In [ ]:
import numpy as np

target_chain_id = 'A'
sampled_seq = esm.inverse_folding.multichain_util.sample_sequence_in_complex(
    model, coords, target_chain_id, temperature=1.)

recovery = np.mean([(a==b) for a, b in zip(native_seqs[target_chain_id], sampled_seq)])
print('Sequence recovery:', recovery)

In [ ]:
# Lower sampling temperature typically results in higher sequence recovery but less diversity

target_chain_id = 'A'
sampled_seq = esm.inverse_folding.multichain_util.sample_sequence_in_complex(
    model, coords, target_chain_id, temperature=1e-6)

recovery = np.mean([(a==b) for a, b in zip(native_seqs[target_chain_id], sampled_seq)])
print('Sequence recovery:', recovery)

## Conditional sequence log-likelihoods for given backbone coordinates

The log-likelihood scores could be used to predict mutational effects. See also our [script](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding#scoring-sequences) for batch scoring,

In [ ]:
target_chain_id = 'A'
target_seq = native_seqs[target_chain_id]
ll_fullseq, ll_withcoord = esm.inverse_folding.multichain_util.score_sequence_in_complex(
    model, alphabet, coords, target_chain_id, target_seq, padding_length=10)
print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

## Masking part of the backbone coordinates
To partially mask backbone coordinates, simply set the masked coordinates to np.inf.

Typically, the sequence perplexity will be higher on masked regions than on unmasked regions.

In [ ]:
target_chain_id = 'A'
target_seq = native_seqs[target_chain_id]

from copy import deepcopy
masked_coords = deepcopy(coords)
masked_coords[target_chain_id][:15] = np.inf # mask the first 15 residues

ll_fullseq, ll_withcoord = esm.inverse_folding.multichain_util.score_sequence_in_complex(
    model, alphabet, masked_coords, target_chain_id, target_seq, padding_length=10)
print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

## Extract encoder output as structure representation
The encoder output may also be used as a representation for the structure.

For a set of input coordinates with L amino acids, the encoder output will have shape L x 512.

In [ ]:
target_chain_id = 'A'
rep = esm.inverse_folding.multichain_util.get_encoder_output_for_complex(model, alphabet, coords, target_chain_id)
len(coords[target_chain_id]), rep.shape